# Interacting With The Map

Up to now we have created varrious widgets and have interacted with the widgets to create Charts and Maps. In this lecture we are going to look at how to use a map as widget itself (through clicks and hover). For this we are going to use a library called ipyleaflet.

## Creating a basic map with ipyleaflet

This could be done with two lines of code using ipyleaflet

In [170]:
from ipyleaflet import Map,Marker,CircleMarker,Circle, Polyline,Polygon,GeoData,GeoJSON,Choropleth
from IPython.display import display,clear_output
import psycopg2
import geopandas as gpd
import pandas as pd
import ipywidgets as widgets
import json
from branca.colormap import linear

In [2]:
myMap = Map(center=(41.504, -81.608), zoom=15)  # I am setting CWRU as the center of my map (latitude,longitude)
display(myMap)

Map(center=[41.504, -81.608], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

### Adding basic geometries to the Map

#### Points

We could use markers, circles, or circle markers to represent points on a map.

##### A Marker

Let us add a single marker above Robbins Building in the map that we have created above. 

In [3]:
#create the marker
marker = Marker(location=(41.50476636811952, -81.60336914291148), draggable=False,title="Welcome to Robbin's building")
#assign it to the map
myMap.add_layer(marker);

If you want to remove the marker from the map you can call the remove layer method on the map with the marker as parameter

In [4]:
#myMap.remove_layer(marker);   #uncomment for removing the marker

##### A Circle Marker

Let us add a circle marker above UH Rainbow Babies in the map that we have created above.

In [5]:
circle_marker = CircleMarker()
circle_marker.location = (41.504278073558474, -81.60568729879611)   #latitude and longitude
circle_marker.radius = 50  #this is in pixels and not in meters
circle_marker.color = "red"
circle_marker.fill_color = "red"
myMap.add_layer(circle_marker);

In [6]:
#myMap.remove_layer(circle_marker);   #uncomment for removing the circle marker

##### A Circle

Let us add a circle above Adelbert hall.

In [7]:
circle = Circle()
circle.location = (41.50484768937726, -81.6078518080495)
circle.radius = 20  #This is in meters and not in pixels 
circle.color = "green"
circle.fill_color = "green"
myMap.add_layer(circle);

In [8]:
#myMap.remove_layer(circle);   #uncomment for removing the circle

Capturing Clicks for Points

We can add click listeners to markers, circle markers or circles easily. Let us add a click listener for our marker

In [9]:
def markerClicked(**changes):
    print(changes)
    
marker.on_click(markerClicked)

Now can you click on the marker and check what is being printed

As you can see the changes parameter is a dictionary with keys event, type, coordinates. We can either use any of this property or do other things with in the function. 

What if you want the marker to get hold of a unique id when clicked (which could be used for further calculations). Let us remove the old marker and create a new marker and then assign a new function to the click listener

In [10]:
myMap.remove_layer(marker);
#create the marker
marker = Marker(location=(41.50476636811952, -81.60336914291148), draggable=False,title="Welcome to Robbin's building")
#assign it to the map
myMap.add_layer(marker);

#now add a new event listener for the marker

def markerClickedNew(val):
    def clicked(**change):
        print ('My id is',val)
    return clicked
    
marker.on_click(markerClickedNew(1))

Now let us do a real world example. Here we are going to show the centroids of the various neighborhoods in NYC as markers. And then when you click on the marker, it will print the total homicide in the neighborhood. 

In [13]:
#get the connection
conn = psycopg2.connect('dbname=nyc user=geospatial password=geospatial2023 host=ghhlibrary')

In [22]:
#query to retrieve the centroids of neighborhoods as points
centroids = gpd.read_postgis(f'select gid,name,st_transform(st_centroid(geom),4326) as geom from nyc_neighborhoods',conn)

E:\sbd3\lib\site-packages\geopandas\io\sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [29]:
#now create the map 

neighbMap = Map(center=(40.69564041317145, -73.85796843331579), zoom=15)  # nyc map
display(neighbMap)

Map(center=[40.69564041317145, -73.85796843331579], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [30]:
# The function that gets called when you click on the marker
def getTotalHomicidesForNeighborhood(gid):
    def clicked(**change):
        #write your query using the gid
        query = f"select count(h.gid) as total from nyc_homicides h,nyc_neighborhoods n where st_contains(n.geom,h.geom) and n.gid={gid}"
        data = pd.read_sql_query(query,conn)
        print ('Total homicides for this county is',data.total.values[0])
    return clicked
#now we will loop through the centroids and then create markers

for idx,row in centroids.iterrows():
    mark= Marker(location=(row.geom.y, row.geom.x), draggable=False,title=row['name'])
    mark.on_click(getTotalHomicidesForNeighborhood(row.gid))
    neighbMap.add_layer(mark);

In [ ]:
As you can see when you click on the marker the print statement get called. We can easily add a label and print the content to the label

In [32]:
#now create the map 

neighbMap2 = Map(center=(40.69564041317145, -73.85796843331579), zoom=15)  # nyc map
label = widgets.Label(value='')
# The function that gets called when you click on the marker
def getTotalHomicidesForNeighborhood(gid):
    def clicked(**change):
        #write your query using the gid
        query = f"select count(h.gid) as total from nyc_homicides h,nyc_neighborhoods n where st_contains(n.geom,h.geom) and n.gid={gid}"
        data = pd.read_sql_query(query,conn)
        label.value= f'Total homicides for this county is {data.total.values[0]}'
    return clicked
#now we will loop through the centroids and then create markers

for idx,row in centroids.iterrows():
    mark= Marker(location=(row.geom.y, row.geom.x), draggable=False,title=row['name'])
    mark.on_click(getTotalHomicidesForNeighborhood(row.gid))
    neighbMap2.add_layer(mark);
    
display(neighbMap2)
display(label)

Map(center=[40.69564041317145, -73.85796843331579], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Label(value='')

Now let us look into other geometries (lines and polygons)

## Lines

Lines can be represented as polyline objects which takes a list of coordinates

In [34]:
line = Polyline(
    locations=[
        [45.51, -122.68],
        [37.77, -122.43],
        [34.04, -118.22]
    ],
    color="green" ,
    fill=False
)

m = Map(center = (42.5, -41), zoom =2)
m.add_layer(line)
display(m)

Map(center=[42.5, -41], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

## Polygons

In [36]:
polygon = Polygon(
    locations=[(42, -49), (43, -49), (43, -48)],
    color="green",
    fill_color="green"
)

m = Map(center=(42.5531, -48.6914), zoom=6)
m.add_layer(polygon);

m

Map(center=[42.5531, -48.6914], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Eventhough we can create points, polygons and polylines as shown above, the real power of ipyleaflet is the ability to map GeoDataFrames (you have already worked with this) and geojsons (will talk about it in the next section). 

## Mapping GeoDataFrame as a Layer

Let us take a simple example. We would like to map the neighborhoods in NYC as polygons itself. 

In [52]:
neighbMap3 = Map(center=(40.69564041317145, -73.85796843331579), zoom=15)  # nyc map
display(neighbMap3)

Map(center=[40.69564041317145, -73.85796843331579], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Now let us retrieve all the neighborhoods

In [53]:
neighbhoods = gpd.read_postgis(f'select gid,name,st_transform(geom,4326) as geom from nyc_neighborhoods',conn)

E:\sbd3\lib\site-packages\geopandas\io\sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [54]:
neighbhoods_data = GeoData(geo_dataframe = neighbhoods,
                   style={'color': 'black', 'fillColor': 'blue', 'opacity':0.05, 'weight':2, 'dashArray':'2', 'fillOpacity':0.3},
                   name = 'neighborhoods')
neighbMap3.add_layer(neighbhoods_data)

As you can see with just a single line of code we can create a map with the neighborhoods Polygon. Now let us add an onclick event to the polygon

In [55]:
def hiClick(**feature):
    print (feature['properties'])
neighbhoods_data.on_click(hiClick)

The properties key in the feature dictionary provides access to all other column values other than geometry (in our case gid and name), which we can use to generate further queries. 

Let us do another interactive example where when you click on a county in Ohio, will create a pandas table of covid totals for different case months.

In [147]:
counties = gpd.read_postgis("select geoid,name,st_transform(geom,4326)  as geom from us_counties where statefp = '39'",conn)
counties

E:\sbd3\lib\site-packages\geopandas\io\sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,geoid,name,geom
0,39035,Cuyahoga,"MULTIPOLYGON (((-81.97116 41.35306, -81.97086 ..."
1,39039,Defiance,"MULTIPOLYGON (((-84.80405 41.40836, -84.80401 ..."
2,39043,Erie,"MULTIPOLYGON (((-82.73571 41.60336, -82.73392 ..."
3,39125,Paulding,"MULTIPOLYGON (((-84.80378 41.14052, -84.80346 ..."
4,39135,Preble,"MULTIPOLYGON (((-84.81512 39.57295, -84.81512 ..."
...,...,...,...
83,39011,Auglaize,"MULTIPOLYGON (((-84.45618 40.68486, -84.39678 ..."
84,39049,Franklin,"MULTIPOLYGON (((-83.26088 40.00281, -83.25855 ..."
85,39111,Monroe,"MULTIPOLYGON (((-81.31824 39.73858, -81.31787 ..."
86,39101,Marion,"MULTIPOLYGON (((-83.42025 40.61398, -83.41983 ..."


In [142]:
ohMap = Map(center=(40.29727789095309, -82.84015921998292), zoom=7)  # oh map
display(ohMap)

Map(center=[40.29727789095309, -82.84015921998292], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [143]:
counties_data = GeoData(geo_dataframe = counties,
                   style={'color': 'black', 'fillColor': 'blue', 'opacity':1, 'weight':3, 'fillOpacity':0},
                   name = 'counties')
ohMap.add_layer(counties_data)

In [144]:
#Let us create an out where we will display the dataframe and then finally wrap everything in an HBox
out = widgets.Output(layout = widgets.Layout(width='40%'))
pd.options.display.max_rows=20         
def countyClicked(**feature):
    geoid = feature['properties']['geoid']
    query = f"select case_month,total from covid_oh_county_totals where county_fips_code ='{geoid}' order by case_month"
    data = pd.read_sql_query(query,conn)
    with out:
        clear_output(wait=True)
        display(data)
counties_data.on_click(countyClicked)
hb = widgets.HBox([ohMap,out])
display(hb)

In [ ]:
Now let us create a chart instead of the dataframe

We will borrow the chart logic from the last lecture

In [145]:
yearMonth = pd.read_sql_query("select distinct case_month from covid_oh_county_totals order by case_month",conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_88060\216084033.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yearMonth = pd.read_sql_query("select distinct case_month from covid_oh_county_totals order by case_month",conn)


In [152]:
ohMap2 = Map(center=(40.29727789095309, -82.84015921998292), zoom=7)  # oh map
out2 = widgets.Output(layout = widgets.Layout(width='45%'))
counties_data2 = GeoData(geo_dataframe = counties,
                   style={'color': 'black', 'fillColor': 'blue', 'opacity':1, 'weight':3, 'fillOpacity':0},
                   name = 'counties')
ohMap2.add_layer(counties_data2)

def countySelected(**feature):
    county = feature['properties']['geoid']
    data = pd.read_sql_query(f"select case_month,total from covid_oh_county_totals where county_fips_code = '{county}' order by case_month",conn)
    data = yearMonth.merge(data,on='case_month',how='left').fillna(0)
    with out2:
        clear_output(wait=True)
        ax = data.plot(logy=True,ylim=((10**0,10**5)),x='case_month',xlabel='Year Month',ylabel='Total Cases',title=f"Covid-19 case distribution for {feature['properties']['name']}",legend=False)
        display(ax.get_figure())
counties_data2.on_click(countySelected)
hb2 = widgets.HBox([ohMap2,out2])
display(hb2)

## Mapping Geojson

Ipyleaflet can map Geojson which is another way of representing spatial data like shapefile. You can easily convert a geodataframe to geojson using the to_json method of the geodataframe

### Creating a geojson file from geodataframe

Let us convert the counties geodataframe to geojson

In [ ]:
countiesGeoJson = counties.to_json(drop_id=True)
countiesGeoJson

As you can see it is countiesGeoJson is a string object in json format which could be converted to a dict using the json package.

In [159]:
countiesGeoJsonDict = json.loads(countiesGeoJson)

## Plotting Geojson

Geojson can be plotted using the GeoJSON object in ipyleaflet. Let us plot our county data as a geojson

In [161]:
ohMap3 = Map(center=(40.29727789095309, -82.84015921998292), zoom=7)  # oh map
geo_json = GeoJSON(
    data=countiesGeoJsonDict,
    style={
        'opacity': 1, 'fillOpacity': 0, 'weight': 2,'color': 'black'
    }
)
ohMap3.add_layer(geo_json)
display(ohMap3)

Map(center=[40.29727789095309, -82.84015921998292], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Creating a click event is same as for geodataframe. 

## Creating Choropleth Maps

Choropleth maps can be created in ipyleaflet using geojson data. Along with the geojson data you would also need to have a dictionary which maps a key in the geojson data to the key in the dictionary and the value will be the value required for plotting.

Let us try an example. In this example we are going to create a choropleth map of the total homicides for different neighborhoods in NYC. 

So we would need

1) The neighborhoods as a geojson contaning the geometry and the primary key for the table (gid).

2) The total number of homicides aggregated for each neighborhood

In [213]:
#the neighborhoods
neighb = gpd.read_postgis("select gid,st_transform(geom,4326) as geom from nyc_neighborhoods",conn)
neighbjson = json.loads(neighb.to_json(drop_id=True))
#Need to have the id one level above
for d in neighbjson['features']:
    d['gid'] = d['properties']['gid']

E:\sbd3\lib\site-packages\geopandas\io\sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [214]:
#aggregated homicide data for each neighborhoods
neighbhomicide = pd.read_sql_query("select n.gid,count(h.gid) as total from nyc_neighborhoods n left join nyc_homicides h on st_contains(n.geom,h.geom) group by n.gid",conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_88060\746618007.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  neighbhomicide = pd.read_sql_query("select n.gid,count(h.gid) as total from nyc_neighborhoods n left join nyc_homicides h on st_contains(n.geom,h.geom) group by n.gid",conn)


In [215]:
#convert to dictionary
valueDict = pd.Series(neighbhomicide.total.values,index=neighbhomicide.gid).to_dict()

In [216]:
neighbMap4 = Map(center=(40.69564041317145, -73.85796843331579), zoom=10)  # nyc map
neighbchoro = Choropleth(
    geo_data=neighbjson,
    choro_data=valueDict,
    key_on='gid',
   colormap=linear.RdYlGn_04,
    border_color='black',
    style={'fillOpacity': 0.8})
neighbMap4.add_layer(neighbchoro)
display(neighbMap4)

Map(center=[40.69564041317145, -73.85796843331579], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Now let us create an interactive map with the year as slider widget and based on the slider we can update the choropleth map.

In [221]:
distinctYears = pd.read_sql_query('select distinct year::text from nyc_homicides order by year',conn)

C:\Users\jxa421\AppData\Local\Temp\ipykernel_88060\2422417265.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  distinctYears = pd.read_sql_query('select distinct year::text from nyc_homicides order by year',conn)


In [228]:
neighbMap5 = Map(center=(40.69564041317145, -73.85796843331579), zoom=10)  # nyc map
currentLayer=None
def yearChanged(change):
    global currentLayer
    neighbhomicide = pd.read_sql_query(f"select n.gid,count(h.gid) as total from nyc_neighborhoods n left join nyc_homicides h on st_contains(n.geom,h.geom) and h.year={change['new']} group by n.gid",conn) 
    valueDict = pd.Series(neighbhomicide.total.values,index=neighbhomicide.gid).to_dict()
    if currentLayer is not None:
        neighbMap5.remove_layer(currentLayer)
    currentLayer = Choropleth(
        geo_data=neighbjson,
        choro_data=valueDict,
        key_on='gid',
        colormap=linear.Reds_05,
        border_color='black',
        style={'fillOpacity': 0.8},
        value_min=0,
        value_max=60
    )
    neighbMap5.add_layer(currentLayer)
    
yearSlider = widgets.SelectionSlider(
    options=distinctYears.year.values,
    value=distinctYears.year.values[0],
    description='Year',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
yearSlider.observe(yearChanged,names='value')
v = widgets.VBox([neighbMap5,yearSlider])
display(v)